In [235]:
from datetime import datetime
import os
import re
import glob
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import folium

In [236]:
link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [237]:
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [238]:
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [239]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,776,589","+7,589","156,277",+862,"1,251,232","+5,879","1,369,080","19,416",,,,,,North America,\n,,
1,,\nSouth America\n,"1,819,580","+3,363","72,629",+47,"1,022,055","+15,307","724,896","12,536",,,,,,South America,\n,,
2,,\nEurope\n,"2,337,164","+10,317","188,185",+256,"1,280,736","+14,637","868,243","6,201",,,,,,Europe,\n,,
3,,\nAsia\n,"1,963,419","+22,745","49,461",+413,"1,253,596","+17,267","660,362","19,259",,,,,,Asia,\n,,
4,,\nAfrica\n,"319,443","+1,122","8,385",+14,"154,059",+817,"156,999",507,,,,,,Africa,\n,,


In [240]:
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"2,389,785","+1,632","122,659",+49,"1,003,320",+391,"1,263,806","16,510","7,221",371,"29,038,933","87,742","330,959,930",North America
1,Brazil,"1,113,606","+2,258","51,407",,"594,104","+14,878","468,095","8,318","5,240",242,"2,510,717","11,814","212,525,202",South America
2,Russia,"599,705","+7,425","8,359",+153,"356,429","+12,013","234,917","2,300","4,109",57,"17,522,752","120,074","145,933,256",Europe
3,India,"441,948","+1,498","14,030",+15,"248,889",+752,"179,029","8,944",320,10,"7,137,716","5,173","1,379,678,183",Asia
4,UK,"305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296","67,878,273",Europe


In [241]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,959,930","2,389,785","+1,632","122,659",+49,"1,003,320",+391,"1,263,806","16,510","7,221",371,"29,038,933","87,742"
1,Brazil,South America,"212,525,202","1,113,606","+2,258","51,407",,"594,104","+14,878","468,095","8,318","5,240",242,"2,510,717","11,814"
2,Russia,Europe,"145,933,256","599,705","+7,425","8,359",+153,"356,429","+12,013","234,917","2,300","4,109",57,"17,522,752","120,074"
3,India,Asia,"1,379,678,183","441,948","+1,498","14,030",+15,"248,889",+752,"179,029","8,944",320,10,"7,137,716","5,173"
4,UK,Europe,"67,878,273","305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296"


In [242]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [243]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['CAR', 'French Guiana', 'Mayotte', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Turks and Caicos', 'Falkland Islands',
       'Montserrat', 'MS Zaandam'], dtype=object)

In [244]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
df_bs = df_bs.replace('', np.nan)
df_bs = df_bs.fillna("Not Available")
df_bs = df_bs.astype(str)
df_bs.drop(columns=['NewCases','NewDeaths','NewRecovered','Serious,Critical'],axis=1,inplace=True)
df_bs.sort_values(by=["Country/Region"],inplace=True)
df_bs.drop(df_bs[df_bs["Continent"]=="Not Available"].index,axis=0,inplace=True)
df_bs = df_bs.reset_index(drop=True)

In [245]:
df_bs["Country/Region"][50]

'Djibouti'

In [246]:
df_bs.to_csv('worldometer_data.csv', index=False)

In [247]:
df=pd.read_csv('worldometer_data.csv')
df=df.set_index(df['Country/Region'])
df.head()

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
Country/Region,,,,,,,,,,,,
Afghanistan,Afghanistan,Asia,38901392,29481,618,9260,19603,758,16,66800,1717,EasternMediterranean
Albania,Albania,Europe,2877855,2047,45,1195,807,711,16,21090,7328,Europe
Algeria,Algeria,Africa,43828594,11920,852,8559,2509,272,19,Not Available,Not Available,Africa
Andorra,Andorra,Europe,77263,855,52,796,7,11066,673,3750,48536,Europe
Angola,Angola,Africa,32830010,186,10,77,99,6,0.3,10000,305,Africa


In [248]:
countries = pd.read_html("https://developers.google.com/public-data/docs/canonical/countries_csv")[0]
countries=countries.set_index(countries['name'])
countries.head()

,country,latitude,longitude,name
name,,,,
Andorra,AD,42.546245,1.601554,Andorra
United Arab Emirates,AE,23.424076,53.847818,United Arab Emirates
Afghanistan,AF,33.939110,67.709953,Afghanistan
Antigua and Barbuda,AG,17.060816,-61.796428,Antigua and Barbuda
Anguilla,AI,18.220554,-63.068615,Anguilla


In [249]:
lat,long=[],[]
m_lat,m_long=[],[]
for country in df['Country/Region']:
    try:
        lat.append((country,countries.loc[country.strip()]['latitude']))
    except:
        m_lat.append(country)
for country in df['Country/Region']:
    try:
        long.append((country,countries.loc[country.strip()]['longitude']))
    except:
        m_long.append(country)

m_lat

['CAR',
 'Cabo Verde',
 'Channel Islands',
 'Congo',
 'Curaçao',
 'Czechia',
 'DRC',
 'Eswatini',
 'Faeroe Islands',
 'Falkland Islands',
 'Ivory Coast',
 'Macao',
 'Myanmar',
 'North Macedonia',
 'Palestine',
 'S. Korea',
 'Saint Martin',
 'Sao Tome and Principe',
 'Sint Maarten',
 'South Sudan',
 'St. Vincent Grenadines',
 'Turks and Caicos',
 'UAE',
 'UK',
 'USA']

In [250]:
s=''
for i in m_lat:
    s+='("{}",),'.format(i)
print(s)

("CAR",),("Cabo Verde",),("Channel Islands",),("Congo",),("Curaçao",),("Czechia",),("DRC",),("Eswatini",),("Faeroe Islands",),("Falkland Islands",),("Ivory Coast",),("Macao",),("Myanmar",),("North Macedonia",),("Palestine",),("S. Korea",),("Saint Martin",),("Sao Tome and Principe",),("Sint Maarten",),("South Sudan",),("St. Vincent Grenadines",),("Turks and Caicos",),("UAE",),("UK",),("USA",),


In [251]:
lat+=[("CAR",6.111),("Cabo Verde",6.5388),("Channel Islands",49.372284),("Congo",-0.2280),("Curaçao",12.1696),("Czechia",49.8175),("DRC",-4.0383),("Eswatini",-26.5225),("Faeroe Islands",61.8926),("Falkland Islands",-51.7963),("Ivory Coast",7.5400),("Macao",22.1987),("Myanmar",21.9162),("North Macedonia",41.6086),("Palestine",31.9522),("S. Korea",35.9078),("Saint Martin",18.0826),("Sao Tome and Principe",0.1864),("Sint Maarten",18.0425),("South Sudan",6.8770),("St. Vincent Grenadines",12.9843),("Turks and Caicos",21.6940),("UAE",23.4241),("UK",55.3781),("USA",37.0902)]

long+=[("CAR",23.9394),("Cabo Verde",-23.0418),("Channel Islands",-2.364351),("Congo",15.8277),("Curaçao",-68.99),("Czechia",15.473),("DRC",-21.7587),("Eswatini",31.4659),("Faeroe Islands",-6.9118),("Falkland Islands",-59.5236),("Ivory Coast",5.5471),("Macao",113.5439),("Myanmar",95.956),("North Macedonia",21.7453),("Palestine",35.2332),("S. Korea",127.7669),("Saint Martin",63.0523),("Sao Tome and Principe",6.6131),("Sint Maarten",-63.0548),("South Sudan",31.307),("St. Vincent Grenadines",-61.2872),("Turks and Caicos",-71.7979),("UAE",53.8478),("UK",-3.436),("USA",-95.7129)]

In [252]:
del df["Country/Region"]
lat.sort()
long.sort()
df.sort_values("Country/Region",inplace=True)
df.head()

,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
Country/Region,,,,,,,,,,,
Afghanistan,Asia,38901392,29481,618,9260,19603,758,16,66800,1717,EasternMediterranean
Albania,Europe,2877855,2047,45,1195,807,711,16,21090,7328,Europe
Algeria,Africa,43828594,11920,852,8559,2509,272,19,Not Available,Not Available,Africa
Andorra,Europe,77263,855,52,796,7,11066,673,3750,48536,Europe
Angola,Africa,32830010,186,10,77,99,6,0.3,10000,305,Africa


In [253]:
count=0
for i,j in zip([x[0] for x in lat],df.index):
    if i!=j:
        count+=1
print(count)

count1=0
for i,j in zip([x[0] for x in long],df.index):
    if i!=j:
        count1+=1
print(count1)

0
0


In [254]:
df['Latitude']=[x[1] for x in lat]
df['Longitude']=[y[1] for y in long]
df.tail()

,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,Latitude,Longitude
Country/Region,,,,,,,,,,,,,
Vietnam,Asia,97317898,349,Not Available,328,21,4,Not Available,275000,2826,WesternPacific,14.058324,108.277199
Western Sahara,Africa,596871,10,1,8,1,17,2,Not Available,Not Available,Africa,24.215527,-12.885834
Yemen,Asia,29805944,967,257,350,360,32,9,120,4,EasternMediterranean,15.552727,48.516388
Zambia,Africa,18366582,1430,11,1194,225,78,0.6,47968,2612,Africa,-13.133897,27.849332
Zimbabwe,Africa,14857190,512,6,64,442,34,0.4,52905,3561,Africa,-19.015438,29.154857


In [255]:
df=df.reset_index()
df.head()

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,Latitude,Longitude
0,Afghanistan,Asia,38901392,29481,618,9260,19603,758,16,66800,1717,EasternMediterranean,33.939110,67.709953
1,Albania,Europe,2877855,2047,45,1195,807,711,16,21090,7328,Europe,41.153332,20.168331
2,Algeria,Africa,43828594,11920,852,8559,2509,272,19,Not Available,Not Available,Africa,28.033886,1.659626
3,Andorra,Europe,77263,855,52,796,7,11066,673,3750,48536,Europe,42.546245,1.601554
4,Angola,Africa,32830010,186,10,77,99,6,0.3,10000,305,Africa,-11.202692,17.873887


In [263]:
map = folium.Map(location = [100,0],zoom_start=2)

for country,lat,long, cases, deaths, recovered, active, tests in zip(list(df["Country/Region"]),list(df['Latitude']),list(df['Longitude']),list(df["TotalCases"]),list(df["TotalDeaths"]),list(df["TotalRecovered"]),list(df["ActiveCases"]),list(df["TotalTests"])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Country : '+country+'</font></striong><br>'+
                            '<strong>Total Cases : '+str(cases)+'</striong><br>'+
                            '<strong>Total Deaths : '+str(deaths)+'</striong><br>'+
                            '<strong>Total Recovered : '+str(recovered)+'</striong><br>'+
                            '<strong>Active Cases : '+str(active)+'</striong><br>'+
                            '<strong>Total Tests : '+str(tests)+'</striong><br>'),max_width=200)).add_to(map)
map